Распаковка датасета

In [ ]:
!unzip "/content/cccompetitionmts.zip" -d "/content/data"

# TF-IDF + RF

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from tqdm.notebook import tqdm
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
import sklearn
import gensim
import gensim.downloader as gensim_api

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

Вспомогательные процедуры для чтения и небольшой аугментации текстов

In [ ]:
def getWords(text):
    text=text[1:-2]
    splitters=list("!#@$%^&*()_+=-<>/\\{}[];:'\"|?,.,~`")
    additional=["\\n", "\\t"]
    text = text.replace('#', " reshetka ")
    text = text.replace('@', " sobaka ")
    text = text.replace('->', " StrelkaTo ")
    text = text.replace('<-', " StrelkaFrom ")
    text = text.replace(';', " tochkaszap ")
    text = text.replace(':', " dvoetoch ")

    for i in additional:
        text = text.replace(i, " ")
    for x in splitters:
        text = text.replace(x, " ")
    while text.find("  ")!=-1:
        text = text.replace("  ", " ")
    return text

In [ ]:
def read_txt(path, prefix):
    text = ""
    with open(prefix+path, mode='r') as f:
        text = getWords(str(f.read()))
    return text

Загрузка датасета

In [ ]:
data = pd.read_csv("/content/data/train_labels.csv", index_col=0)
tqdm.pandas()
data['text']=data.progress_apply(lambda row: read_txt(row['id'], "/content/data/train/"), axis = 1)

  0%|          | 0/3297 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

Делим датасет на обучающую и тестовую выборки

In [ ]:
X_train, X_test , y_train, y_test = sklearn.model_selection.train_test_split(data['text'].values,data['label'].values,test_size=0.15,random_state=42,stratify=data['label'].values)

Обучеаем модель

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
classifier.fit(tfidf_train_vectors,y_train)

In [ ]:
y_pred = classifier.predict(tfidf_test_vectors)

Метрики полученной модели

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        42
           1       1.00      1.00      1.00        45
           2       1.00      1.00      1.00         8
           3       0.99      1.00      0.99        95
           4       1.00      1.00      1.00        71
           5       1.00      1.00      1.00        56
           6       1.00      1.00      1.00        15

    accuracy                           1.00       332
   macro avg       1.00      1.00      1.00       332
weighted avg       1.00      1.00      1.00       332



Загружаем тестовую часть

In [ ]:
datae = pd.read_csv("/content/data/test_ids.csv", index_col=0)

tqdm.pandas()
datae['text']=datae.progress_apply(lambda row: read_txt(row['id'], "/content/data/test/"), axis = 1)
datae.tail()

  0%|          | 0/823 [00:00<?, ?it/s]

,id,text
818,script_14110.txt,package lyons entity tochkaszap Gsales g xb9 ...
819,script_14111.txt,using System Diagnostics tochkaszap namespace...
820,script_14112.txt,reshetka Extract Data Information From Argume...
821,script_14113.txt,reshetka LANGUAGE FlexibleInstances reshetka ...
822,script_14114.txt,package com loper7 datepicker import android ...


Запускаем модель на тестовой выборке

In [ ]:
eval_vectors = tfidf_vectorizer.transform(datae['text'])
#e_pred = catboost_model.predict(eval_vectors)

e_pred = classifier.predict(eval_vectors)

Записываем ответ

In [ ]:
submission=pd.read_csv("/content/data/test_ids.csv", index_col=0)
submission['label']=e_pred.squeeze().tolist()
submission.reset_index(drop=True).to_csv("output.csv", index=False)

# Catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import Pool, CatBoostClassifier
catboost_model = CatBoostClassifier(learning_rate=0.03,task_type="GPU",
                           loss_function='MultiClass')

In [ ]:
cb_val_dataset = Pool(data=tfidf_test_vectors,
                    label=y_test)
catboost_model.fit(tfidf_train_vectors,
          y_train,
          eval_set=cb_val_dataset)

In [ ]:
y_pred = catboost_model.predict(tfidf_test_vectors)

In [ ]:
print(classification_report(y_train,y_predtrain))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       377
           1       1.00      1.00      1.00       400
           2       1.00      1.00      1.00        68
           3       1.00      1.00      1.00       857
           4       1.00      1.00      1.00       639
           5       1.00      1.00      1.00       503
           6       1.00      1.00      1.00       137

    accuracy                           1.00      2981
   macro avg       1.00      1.00      1.00      2981
weighted avg       1.00      1.00      1.00      2981



In [ ]:
e_pred.squeeze()

array([5, 4, 1, 3, 1, 1, 3, 3, 6, 0, 4, 3, 5, 5, 0, 1, 5, 4, 1, 0, 5, 1,
       0, 3, 5, 1, 5, 3, 5, 4, 4, 3, 5, 0, 3, 4, 3, 5, 3, 1, 4, 1, 3, 5,
       1, 5, 5, 5, 5, 6, 0, 3, 3, 1, 3, 1, 3, 5, 3, 3, 3, 3, 0, 2, 1, 3,
       3, 6, 3, 3, 4, 5, 0, 3, 3, 3, 3, 0, 5, 3, 5, 0, 5, 3, 4, 6, 5, 5,
       3, 1, 1, 3, 5, 4, 3, 3, 4, 6, 3, 3, 4, 3, 4, 5, 3, 1, 3, 4, 3, 4,
       3, 0, 1, 4, 0, 3, 5, 5, 3, 3, 4, 3, 4, 4, 4, 1, 5, 1, 1, 1, 3, 5,
       5, 3, 4, 3, 4, 1, 3, 4, 3, 3, 3, 5, 3, 5, 0, 1, 1, 6, 3, 1, 5, 1,
       1, 5, 3, 3, 0, 5, 0, 5, 3, 1, 2, 4, 3, 1, 1, 5, 4, 5, 5, 5, 1, 3,
       3, 5, 5, 1, 3, 0, 1, 4, 4, 5, 4, 0, 0, 3, 5, 5, 5, 0, 4, 3, 4, 4,
       1, 3, 4, 4, 4, 5, 0, 4, 4, 5, 4, 5, 0, 3, 3, 4, 4, 5, 4, 3, 0, 0,
       3, 6, 3, 3, 4, 1, 3, 5, 3, 1, 5, 4, 0, 1, 4, 4, 1, 0, 3, 4, 1, 4,
       3, 3, 3, 5, 0, 3, 4, 3, 5, 3, 6, 3, 3, 5, 3, 1, 0, 6, 4, 3, 5, 3,
       3, 4, 1, 4, 1, 6, 1, 0, 5, 1, 0, 4, 1, 3, 4, 2, 4, 5, 0, 3, 3, 3,
       0, 1, 0, 4, 3, 6, 0, 4, 3, 4, 3, 3, 3, 3, 3,

In [ ]:
submission=pd.read_csv("/content/data/test_ids.csv", index_col=0)
submission['label']=e_pred.squeeze().tolist()
submission.reset_index(drop=True).to_csv("output.csv", index=False)
submission

,id,label
0,script_13292.txt,5
1,script_13293.txt,4
2,script_13294.txt,1
3,script_13295.txt,3
4,script_13296.txt,1
...,...,...
818,script_14110.txt,4
819,script_14111.txt,0
820,script_14112.txt,6
821,script_14113.txt,3
